# Initialization

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [3]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 11
ROUNDS = 11
OTHER_DATE = '2024-03-23'
_RESETS = False
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:05.295665+00:00.


# Get IQ data

In [4]:
SHOTS = int(1e2)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]
# NOISE_LIST = None

IQ_data= simulator.generate_IQ(SHOTS, noise_list=NOISE_LIST)

# Decode IQ data

In [5]:
import numpy as np
import cpp_soft_info
from Scratch import load_calibration_memory

relError = 1
num_points = 51

bandwidths = np.linspace(0.1, 0.5, 20)

bandwidths = [0.2]

all_memories = load_calibration_memory(provider, tobecalib_backend=DEVICE, other_date=OTHER_DATE)
kde_dict = cpp_soft_info.get_KDEs(all_memories, bandwidths, relError=relError, absError=-1, num_points=num_points)

Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:05.295665+00:00.


# Convertor decoding

In [6]:
from soft_info import inv_qubit_mapping
invert_qubit_mapping = inv_qubit_mapping(simulator.qubit_mapping)

In [7]:
pSoft, countMat = cpp_soft_info.iqConvertor(IQ_data, invert_qubit_mapping, kde_dict, relError, -1)

In [8]:
print(np.mean(pSoft))

0.02588702577960091


# Stim decoding

In [9]:
from soft_info import RepetitionCodeStimCircuit

XBASIS = False
NOISE_LIST_2 = [3e-2, 1e-3, 1e-2, 3e-2, 3e-2, 0.5e-2, 0.5e-2]
SUBSAMPLING = False

code = RepetitionCodeStimCircuit(DISTANCE, ROUNDS, XBASIS, _RESETS, NOISE_LIST_2)
stim_circ = code.circuits[str(LOGICAL)]

In [10]:
result = cpp_soft_info.decodeStimSoft(stim_circ, countMat, pSoft, ROUNDS, int(LOGICAL), _RESETS, False)
print("num_errors KDE:", result.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)

Timings for shot 0 (seconds):
	Copying: num_errors KDE: 0 out of 100 shots for _RESETS = False
6.15e-05
	Softening: 0.000324292
	Error Model Creation: 0.00553662
	Matching Graph Creation: 0.000536875
	Det Syndromes: 3.7084e-05
	Detection Events: 3.6667e-05
	Decoding: 9.1583e-05
	Saving: 0
Timings for shot 1 (seconds):
	Copying: 2.8417e-05
	Softening: 0.000276375
	Error Model Creation: 0.00528825
	Matching Graph Creation: 0.000488167
	Det Syndromes: 3.2e-05
	Detection Events: 4.25e-06
	Decoding: 7.5875e-05
	Saving: 0
Timings for shot 2 (seconds):
	Copying: 2.6333e-05
	Softening: 0.000267125
	Error Model Creation: 0.00509671
	Matching Graph Creation: 0.000481125
	Det Syndromes: 2.9917e-05
	Detection Events: 3.792e-06
	Decoding: 8.0916e-05
	Saving: 4.2e-08
Timings for shot 3 (seconds):
	Copying: 2.1333e-05
	Softening: 0.00025725
	Error Model Creation: 0.00547071
	Matching Graph Creation: 0.000517292
	Det Syndromes: 4.2459e-05
	Detection Events: 5.375e-06
	Decoding: 8.825e-05
	Saving: 4.1e

In [11]:
result = cpp_soft_info.decodeConvertorSoft(stim_circ.detector_error_model(), countMat, pSoft, ROUNDS, int(LOGICAL), _RESETS, False)

# Using cpp kde

In [25]:
import stim
import pymatching

# circ = simulator.stim_circ
circ = stim_circ 
model = circ.detector_error_model(decompose_errors=False)


In [26]:
_DETAILED = False

matching = pymatching.Matching.from_detector_error_model(model)
result = cpp_soft_info.decode_IQ_fast(model, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED, nb_intervals=-1)

print("num_errors IQ:", result.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)

AttributeError: 'RepCodeIQSimulator' object has no attribute 'grid_dict'

In [27]:
_DETAILED = False

relError = 1
absError = -1

# relError = 1
# absError = 10

result_kde = cpp_soft_info.decode_IQ_kde(model, IQ_data, ROUNDS, int(LOGICAL), _RESETS, 
                                         simulator.qubit_mapping, kde_dict, _DETAILED, relError=relError, absError=absError,
                                         nb_intervals=-1)

print("num_errors KDE:", result_kde.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)

num_errors KDE: 0 out of 100 shots for _RESETS = False


In [28]:
p_meas = -1
# p_meas = 7e-2 

result_informed = cpp_soft_info.decode_IQ_shots_flat_informed(model, IQ_data[:],
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict,
                                           p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=_DETAILED, _ntnn_edges = True)

print("num_errors informed:", result_informed.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)
         
# takes 1s

AttributeError: 'RepCodeIQSimulator' object has no attribute 'grid_dict'